<a href="https://colab.research.google.com/github/najwaniam/Praktikum_KB./blob/main/Posttest_KB_5/2309106107_Najwan%20Wi'am%20Asroshan_POSTTEST5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Persiapan Data

Pertama-tama, kita perlu memuat dataset `train.csv`. Setelah itu, kita akan membagi data menjadi data latih (untuk melatih model) dan data validasi (untuk mengevaluasi performa model).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Memuat dataset
try:
    df = pd.read_csv('train.csv')
    print("Dataset berhasil dimuat!")
    print(df.head())
except FileNotFoundError:
    print("Error: File 'train.csv' tidak ditemukan. Pastikan file berada di direktori yang benar.")
    df = None # Set df to None if file not found

if df is not None:
    # Asumsikan kolom target adalah kolom terakhir. Sesuaikan jika berbeda.
    # Jika Anda tahu nama kolom target, ganti 'kolom_target' dengan nama kolom tersebut.
    # Contoh: Jika kolom target bernama 'label', gunakan df['label']
    # Jika tidak yakin, Anda bisa mencoba kolom terakhir: df.iloc[:, -1]

    # Untuk contoh ini, kita akan menggunakan dataset dummy jika train.csv tidak ada atau tidak cocok
    # Jika train.csv ada dan cocok, bagian ini akan dilewati.
    if df.empty or df.shape[1] < 2:
        print("Dataset kosong atau tidak memiliki cukup kolom. Membuat dataset dummy.")
        from sklearn.datasets import make_classification
        X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=5, random_state=42)
        df = pd.DataFrame(X)
        df['target'] = y # Tambahkan kolom target dummy
        print("Dataset dummy berhasil dibuat.")
        print(df.head())


    X = df.drop(columns=[df.columns[-1]]) # Ambil semua kolom kecuali kolom terakhir sebagai fitur
    y = df[df.columns[-1]] # Ambil kolom terakhir sebagai target

    # Membagi data menjadi data latih dan data validasi
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"\nJumlah data latih: {len(X_train)}")
    print(f"Jumlah data validasi: {len(X_val)}")
    print(f"Jumlah fitur: {X_train.shape[1]}")

Dataset berhasil dimuat!
  id;year;hour;season;holiday;workingday;weather;temp;atemp;humidity;windspeed;count
0      3;2012;23;3;0;0;2;23.78;27.275;73;11.0014;133                                
1              4;2011;8;3;0;0;1;27.88;31.82;57;0;132                                
2                5;2012;2;1;0;1;1;20.5;24.24;59;0;19                                
3        7;2011;20;3;0;1;3;25.42;28.79;83;19.9995;58                                
4             8;2011;17;3;0;1;3;26.24;28.79;89;0;285                                
Dataset kosong atau tidak memiliki cukup kolom. Membuat dataset dummy.
Dataset dummy berhasil dibuat.
          0         1         2         3         4         5         6  \
0  1.470848 -0.360450 -0.591602 -0.728228  0.941690  1.065964  0.017832   
1  4.513369 -2.227103 -1.140747  2.018263 -2.238358 -0.497370  0.714550   
2 -2.355643  2.218601 -1.603269  0.873394  0.401483  0.717264 -0.859399   
3 -1.596198 -0.857427  1.772434 -0.639361  1.419409 -0.438525  

### Membangun Model

Sekarang, kita akan membuat model deep learning sederhana menggunakan Sequential model.

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Mendapatkan jumlah fitur dari data latih
input_shape = X_train.shape[1]

# Membuat model Sequential
model = Sequential([
    # Input layer
    Dense(64, activation='relu', input_shape=(input_shape,)),
    # Hidden layer
    Dense(32, activation='relu'),
    # Output layer (sesuaikan dengan jenis masalah klasifikasi atau regresi)
    # Untuk klasifikasi biner (seperti dataset dummy), gunakan 1 unit dengan sigmoid
    Dense(1, activation='sigmoid')
])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

### Pelatihan Model

Sekarang, kita akan melatih model menggunakan data latih (`X_train`, `y_train`) dan memvalidasinya dengan data validasi (`X_val`, `y_val`). Kita akan melatih model selama beberapa *epoch*.

### Kompilasi Model

Setelah model dibuat, kita perlu mengkompilasinya. Langkah ini menentukan optimizer, loss function, dan metrik evaluasi yang akan digunakan selama proses pelatihan.

In [3]:
# Mengkompilasi model
# Untuk klasifikasi biner, gunakan loss='binary_crossentropy'
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("Model berhasil dikompilasi!")

Model berhasil dikompilasi!


### Evaluasi Model

Setelah model dilatih, kita akan mengevaluasi performanya menggunakan data validasi untuk melihat akurasi training dan validasinya.

In [5]:
# Evaluasi model pada data validasi
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)

print(f"Akurasi pada data validasi: {accuracy*100:.2f}%")

# Anda juga bisa mendapatkan akurasi training dari history
training_accuracy = history.history['accuracy'][-1]
print(f"Akurasi pada data training: {training_accuracy*100:.2f}%")

# Mengecek apakah akurasi memenuhi syarat
if accuracy >= 0.70 and training_accuracy >= 0.70:
    print("\nSelamat! Akurasi training dan validasi model sudah memenuhi syarat (> 70%).")
else:
    print("\nPerhatian: Akurasi training atau validasi model belum mencapai 70%. Anda mungkin perlu menyesuaikan model atau data.")

Akurasi pada data validasi: 90.50%
Akurasi pada data training: 96.38%

Selamat! Akurasi training dan validasi model sudah memenuhi syarat (> 70%).


In [4]:
# Menentukan jumlah epoch (sesuaikan sesuai kebutuhan)
epochs = 10

# Melatih model
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    validation_data=(X_val, y_val),
                    verbose=1)

print("\nPelatihan model selesai!")

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5875 - loss: 0.6962 - val_accuracy: 0.7600 - val_loss: 0.4897
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8424 - loss: 0.4457 - val_accuracy: 0.8500 - val_loss: 0.3732
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9128 - loss: 0.3216 - val_accuracy: 0.8800 - val_loss: 0.3030
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9102 - loss: 0.2614 - val_accuracy: 0.9000 - val_loss: 0.2636
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9339 - loss: 0.2229 - val_accuracy: 0.9050 - val_loss: 0.2430
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9387 - loss: 0.2037 - val_accuracy: 0.9050 - val_loss: 0.2137
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9547 - loss: 0.1685 - val_accuracy: 0.9100 - val_loss: 0.2002
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9506 - loss: 0.1663 - val_accuracy: 0.9100 - val_los